In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(googlesheets)
}))

gs_token <- "/home/users/ytanigaw/.googlesheets_token.rds"
gs_auth(token = gs_token)



Auto-refreshing stale OAuth token.



In [2]:
gwas_catalog_full_fs <- 'GWAS-catalog-dump/gwas-association-downloaded_2020-08-18-pubmedId_*.tsv'
gwas_catalog_filtered <- 'GWAS-catalog-dump/gwas-association-downloaded_2020-08-18.filtered.traits.tsv'


In [3]:
'https://docs.google.com/spreadsheets/d/1GsExuaVaf8-t0HTfcuxD-tS5pzemUJUI8WWrEBT6QuA' %>%
gs_url() %>% 
gs_read(
    ws = 'GWAS-catalog-traits'
) %>%
rename('MAPPED_TRAIT'='#MAPPED_TRAIT') -> gwas_catalog_traits


Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.

Putative key: 1GsExuaVaf8-t0HTfcuxD-tS5pzemUJUI8WWrEBT6QuA

Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”
Sheet successfully identified: "GWAS-catalog-traits"

Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”
Accessing worksheet titled 'GWAS-catalog-traits'.

Parsed with column specification:
cols(
  `#MAPPED_TRAIT` = col_character(),
  n = col_double(),
  `is_relevant?` = col_character(),
  Abbreviation = col_character()
)



In [4]:
Sys.glob(gwas_catalog_full_fs) %>%
lapply(function(in_f){
    in_f %>%
    fread(colClasses = 'character')
}) %>% 
bind_rows() -> gwas_catalog_full


In [5]:
gwas_catalog_full %>%
filter(
    MAPPED_TRAIT %in% (
        gwas_catalog_traits %>% 
        filter(`is_relevant?` =='Y') %>%
        pull(MAPPED_TRAIT)        
    ),
    as.numeric(`P-VALUE`) <= 5e-8
) -> gwas_catalog_filtered_df


In [6]:
gwas_catalog_filtered_df %>% colnames()

[1] "DATE ADDED TO CATALOG"      "PUBMEDID"                  
 [3] "FIRST AUTHOR"               "DATE"                      
 [5] "JOURNAL"                    "LINK"                      
 [7] "STUDY"                      "DISEASE/TRAIT"             
 [9] "INITIAL SAMPLE SIZE"        "REPLICATION SAMPLE SIZE"   
[11] "REGION"                     "CHR_ID"                    
[13] "CHR_POS"                    "REPORTED GENE(S)"          
[15] "MAPPED_GENE"                "UPSTREAM_GENE_ID"          
[17] "DOWNSTREAM_GENE_ID"         "SNP_GENE_IDS"              
[19] "UPSTREAM_GENE_DISTANCE"     "DOWNSTREAM_GENE_DISTANCE"  
[21] "STRONGEST SNP-RISK ALLELE"  "SNPS"                      
[23] "MERGED"                     "SNP_ID_CURRENT"            
[25] "CONTEXT"                    "INTERGENIC"                
[27] "RISK ALLELE FREQUENCY"      "P-VALUE"                   
[29] "PVALUE_MLOG"                "P-VALUE (TEXT)"            
[31] "OR or BETA"                 "95% CI (TEXT)"             
[33] "PLATFORM [SNPS PASSING QC]" "CNV"                       
[35] "MAPPED_TRAIT"               "MAPPED_TRAIT_URI"          
[37] "STUDY ACCESSION"            "GENOTYPING TECHNOLOGY"

In [7]:
gwas_catalog_filtered_df %>% dim()

[1] 1258   38

In [8]:
gwas_catalog_filtered_df %>%
rename('#DATE ADDED TO CATALOG' = 'DATE ADDED TO CATALOG') %>%
fwrite(gwas_catalog_filtered, sep='\t', na = "NA", quote=F)
